In [21]:
import openai

In [22]:
client = openai.Client()

In [23]:
assistant = client.beta.assistants.create(
    name="Tutor de Tecnologia",
    instructions="Você é um tutor sobre assuntos relacionados a tecnologia",
    tools=[{"type":"code_interpreter"}],
    model="gpt-3.5-turbo-0125"
)

In [24]:
pergunta = """
Em um sistema de cache com mapeamento direto, dado uma memória principal de um cache 
de 256 KB e blocos de 64 bytes:
- Quantas linhas existem na cache?
- Onde o bloco de endereço 0x1A23C será mapeado na cache?
"""

In [25]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
    
)

In [26]:
#Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
    
)


In [27]:
run

Run(id='run_oYZs2woCIBXAgb6uIMcSY30h', assistant_id='asst_KJdqBLxAoVqBuuXW7nPgRcKP', cancelled_at=None, completed_at=None, created_at=1746809452, expires_at=1746810052, failed_at=None, incomplete_details=None, instructions='Nome de usuário premium', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo-0125', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_ycEKiXcvVaEUAtGD2CBfPg5z', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)

In [28]:
#Aguarda a thread rodar
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [29]:
#Verificar a respota
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"ERRO {run.status}")    
    

SyncCursorPage[Message](data=[Message(id='msg_m9AEnhPsNaKWFYD8l9HI0qcx', assistant_id='asst_KJdqBLxAoVqBuuXW7nPgRcKP', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O endereço 0x1A23C convertido para binário é 11010001000111100.\n\nO bloco de endereço 0x1A23C será mapeado na linha de índice 1672 da cache.\n\nSe precisar de mais alguma coisa, estarei por aqui.'), type='text')], created_at=1746809466, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_oYZs2woCIBXAgb6uIMcSY30h', status=None, thread_id='thread_ycEKiXcvVaEUAtGD2CBfPg5z'), Message(id='msg_DjAUFXh3TR4aYzPwwVj7whUb', assistant_id='asst_KJdqBLxAoVqBuuXW7nPgRcKP', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='A cache com mapeamento direto possui 4096 linhas.\n\nAgora, vamos determinar onde o bloco de endereço 0x1A23C será mapeado na cache. Para isso, primeiro precisamos con

In [30]:
print(mensagens.data[0].content[0].text.value)

O endereço 0x1A23C convertido para binário é 11010001000111100.

O bloco de endereço 0x1A23C será mapeado na linha de índice 1672 da cache.

Se precisar de mais alguma coisa, estarei por aqui.


In [31]:
# Analisando espaços do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [33]:
for step in run_steps.data[::-1]:
    print(f"\n====Step{step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("=" *10)
            if step.step_details.type == "message_creation":
                message = client.beta.threads.messages.retrieve(
                  thread_id=thread.id,
                  message_id=step.step_details.message_creation.message_id
                )
                
                print(message.content[0].text.value)


====Stepmessage_creation

====Steptool_calls
# Tamanho da cache em KB e tamanho do bloco em bytes
cache_size_kb = 256
block_size_bytes = 64

# Convertendo o tamanho da cache para bytes
cache_size_bytes = cache_size_kb * 1024

# Calculando o número de linhas na cache
num_lines_cache = cache_size_bytes / block_size_bytes

num_lines_cache

====Stepmessage_creation

====Steptool_calls
import math

# Endereço do bloco em hexadecimal
address = 0x1A23C

# Convertendo o endereço para binário
address_binary = bin(address)[2:]

# Calculando o tamanho do bloco em bits (log2 do tamanho do bloco)
block_size_bits = int(math.log2(block_size_bytes))

# Calculando o número de blocos na cache (tamanho da cache dividido pelo tamanho do bloco)
num_blocks_cache = cache_size_bytes / block_size_bytes

# Calculando o índice do bloco na cache
block_index = address // block_size_bytes % num_blocks_cache  # Índice do bloco na cache

address_binary, block_index

====Stepmessage_creation
